In [1]:
def simulate(bodies, step = 1, period = 365, method = 'euler'):
    step = 24*step*3600 # Total of seconds for each step: 24*3600 = one day
    count = 0
    pos = {}
    
    def future_step(atual, k, stepper):
        return atual + k*stepper
    
    for body in bodies:
        pos[body.name] = []
        ## Routine that resets body position state in each simulation
        body.p_x = body.p_x0
        body.p_y = body.p_y0
        body.v_x = body.v_x0
        body.v_y = body.v_y0
        
    while count < period:
        acc = {}
        for body in bodies:
            acc_x = acc_y = 0
            for other in bodies:
                if body is other:
                    continue
                a_x, a_y = body.acceleration(other)
                acc_x += a_x
                acc_y += a_y
            acc[body] = (acc_x, acc_y)

        for body in bodies:
            pos[body.name].append((body.p_x,body.p_y))
            
            a_x, a_y = acc[body]               
            if method == 'euler-cromer':
                
                body.v_x += a_x * step
                body.v_y += a_y * step

                body.p_x += body.v_x * step
                body.p_y += body.v_y * step
                
            elif method == 'rk4':
                
                k1_px = body.v_x * step
                k1_py = body.v_y * step
                k1_vx = a_x * step
                k1_vy = a_y * step
                
                for other in bodies:
                    if body is other:
                        continue
                    a_x, a_y = body.acceleration(other, method = 'rk4', k = [k1_px/2,k1_py/2])
                
                k2_px = (body.v_x + k1_vx) * step
                k2_py = (body.v_y + k1_vx) * step
                k2_vx = a_x * step
                k2_vy = a_y * step
                
                for other in bodies:
                    if body is other:
                        continue
                    a_x, a_y = body.acceleration(other, method = 'rk4', k = [k2_px/2,k2_py/2])
                    
                k3_px = (body.v_x + k2_vx) * step
                k3_py = (body.v_y + k2_vx) * step
                k3_vx = a_x * step
                k3_vy = a_y * step
                                
                for other in bodies:
                    if body is other:
                        continue
                    a_x, a_y = body.acceleration(other, method = 'rk4', k = [k3_px,k3_py])
                    
                k4_px = (body.v_x + k3_vx) * step
                k4_py = (body.v_y + k3_vx) * step
                k4_vx = a_x * step
                k4_vy = a_y * step
                    
                body.p_x += (k1_px + 2 * (k2_px + k3_px) + k4_px)/6
                body.p_y += (k1_py + 2 * (k2_py + k3_py) + k4_py)/6
                
                body.v_x += (k1_vx + 2 * (k2_vx + k3_vx) + k4_vx)/6
                body.v_y += (k1_vy + 2 * (k2_vy + k3_vy) + k4_vy)/6
                
            else:
                body.p_x += body.v_x * step
                body.p_y += body.v_y * step
                       
                body.v_x += a_x * step
                body.v_y += a_y * step

        count += 1
    return pos